<a href="https://colab.research.google.com/github/estskyway/study_data_analytics/blob/main/codes/NLP/healthapp_review_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer : https://heytech.tistory.com/401

In [ ]:
# !pip install python-mecab-ko pyLDAvis

In [ ]:
import numpy as np

## 전처리

In [ ]:
import pandas as pd
df_raw = pd.read_excel('./dataset_raw.xlsx')
df_raw[:2]

,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5


### 결측치 처리

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


In [ ]:
df_raw_dropna = df_raw.dropna()
df_raw_dropna.isnull().sum()

app       0
review    0
rating    0
dtype: int64

### 자연어 전처리

#### 특정(상업용) 앱 리스트에서 제거

In [ ]:
remove_app_list = pd.read_excel('./remove_app_list.xlsx')
remove_app_list[:5]

,app
0,캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1,만보기 - 걸음 계산기
2,딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3,타임캐시 – 돈버는 어플
4,돈버는어플 - 캐시런


In [ ]:
remove_app_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app     106 non-null    object
dtypes: object(1)
memory usage: 976.0+ bytes


In [ ]:
for remove_app in remove_app_list['app']:
  try:
    df_removed_app = df_raw_dropna[df_raw_dropna['app'] != remove_app]
  except:
    pass

In [ ]:
df_removed_app = df_removed_app[500:9000]

### 형태소 적용과 불용어 처리

In [ ]:
# 불용어 사전 불러오기
df_stopwords = pd.read_excel('./stopword_list.xlsx')
df_stopwords[:5]

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [ ]:
from mecab import MeCab
mecab = MeCab()
stopwords = df_stopwords['stopword'].values
# 토크나이징 함수 정의(한자 이상 적용. 불용어 삭제)
def tokenizer(raw, stopword=stopwords):
    result_list = list()
    for word in mecab.nouns(raw) :
        if len(word) > 1 and word not in stopword :
            result_list.append(word)
    # 일부 경우(토큰 갯수 제한)
    # if len(result_list) <= 3 or len(result_list) < 12 :
    #   result_list
    return ' '.join(result_list) # return str

In [ ]:
# mecab.nouns('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

In [ ]:
# 형태분석기 적용된 함수 실행
tokenizer('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

'운동'

In [ ]:
df_removed_app['tokenized_review'] = df_removed_app['review'].apply(tokenizer)
df_removed_app[:2]

,app,review,rating,tokenized_review,replaced_review


#### 유사어 대치

In [ ]:
df_replace_list = pd.read_excel('./replace_list.xlsx')
df_replace_list[:4]

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰
3,G7,스마트폰


In [ ]:
def replace_word(tokenized_review):
  replaced_review = str()
  for i in range(len(df_replace_list['before_replacement'])):
    try:
      # 치환할 단어가 있는 경우에만 데이터 치환 수행
      if df_replace_list['before_replacement'][i] in tokenized_review:
        replaced_review = tokenized_review.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
      else :
        replaced_review = tokenized_review
    except Exception as e:
      print(f"Error 발생 / 에러명: {e}")
      replaced_review = tokenized_review
  return replaced_review

In [ ]:
replace_word('S헬스 한글 LG V10업데이트 완벽')

'S헬스 한글 LG V10업데이트 완벽'

In [ ]:
replace_word('한글 업데이트 완벽')

'한글 업데이트 완벽'

In [ ]:
replace_word('')

''

In [ ]:
df_removed_app['replaced_review'] = df_removed_app['tokenized_review'].apply(replace_word)
df_removed_app[:2]

,app,review,rating,tokenized_review,replaced_review


In [ ]:
df_removed_app.to_csv('./preprocess_reviews.csv')

### 부정과 긍정 댓글 분리

In [ ]:
condition_positive = df_removed_app['rating'] >= 3 # 긍정
condition_negative = df_removed_app['rating'] < 3

In [ ]:
df_reviews_positive = df_removed_app[condition_positive]
df_reviews_negative = df_removed_app[condition_negative]

In [ ]:
df_reviews_positive[:3]

,app,review,rating,tokenized_review,replaced_review


In [ ]:
df_reviews_negative[:3]

,app,review,rating,tokenized_review,replaced_review


In [ ]:
df_reviews_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               0 non-null      object
 1   review            0 non-null      object
 2   rating            0 non-null      int64 
 3   tokenized_review  0 non-null      object
 4   replaced_review   0 non-null      object
dtypes: int64(1), object(4)
memory usage: 0.0+ bytes


In [ ]:
df_reviews_negative.dropna(inplace=True)
df_reviews_positive.dropna(inplace=True)

In [ ]:
df_reviews_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               0 non-null      object
 1   review            0 non-null      object
 2   rating            0 non-null      int64 
 3   tokenized_review  0 non-null      object
 4   replaced_review   0 non-null      object
dtypes: int64(1), object(4)
memory usage: 0.0+ bytes


### 긍정, 부정으로 전처리한 것을 csv 파일로 저장

In [ ]:
df_reviews_positive.to_csv('reviews_positive.csv')
df_reviews_negative.to_csv('reviews_negative.csv')

## LDA 분석